#### Introduction to Statistical Learning, Exercise 3.4

__Please do yourself a favour and only look at the solutions after you honestly tried to solve the exercises.__

# Varying Noise in Simulated Data

We will simulate a data set and investigate how things change with varying parameters of the simulation. This allows us to evaluate our models against the simulation truth.

We will use `numpy`'s random generator facilities for the simulation. If you want reproducible results you should set the random seed explicitly like this at the beginning (the actual seed value does not matter):

```python
np.random.seed(seed=123)
```

 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from islpwf import datasets, utils, lmplots
sns.set()
%matplotlib inline

### A. Simulate the Feature

Using the `np.random.normal()` function, create a a vector `x` containing a 100 observations drawn from a $N(0, 1)$ distribution. This represents the feature $X$.

In [ ]:
np.random.seed(seed=123)  # we only need to do this once at the start
x = np.random.normal(size=100)
x

### B. Simulate the Noise

Using the `np.random.normal()` function, create a a vector `eps` containing a 100 observations drawn from a $N(0, 0.5)$ distribution. That is, a normal distribution with mean 0 and standard deviation 0.5. This represents the noise $\epsilon$.

In [ ]:
eps = np.random.normal(0, 0.5, size=100)
eps

### C. Simulate the Response

Using the vectors `x` and `eps`, generate a vector `y` according to the following model:

$$ Y = -1 + 0.5 X + \epsilon $$

This represents the response $Y$. What is the length of the vector `y`? What are the values of $\beta_0$ and $\beta_1$ in this model?

In [ ]:
y = -1 + 0.5 * x + eps 
y

In [ ]:
y.size, y.shape[0]

$\beta_0 = -1$ and $\beta_1 = 0.5$

### D. Plot the y-x Relationship

Create a scatter plot of `y` versus `x`. Comment on what you observe.

In [ ]:
ax = sns.scatterplot(x, y)

There is clearly a linear relationship in the simulated data. The simulated noise is also clearly visible.

### E. Linear Model Fit

Fit at least squares model to the data with `y` as the response and `x` as the predictor. Comment on what you observe. How are the estimated parameters $\hat{\beta}_0$ and $\hat{\beta}_1$ compare to the true values $\beta_0$ and $\beta_1$?

Hint: you can do this directly with `sm.OLS()` but creating a data frame first and using `smf.ols()` might be more convenient.

In [ ]:
data = pd.DataFrame({'y': y, 'x':x})
data.head()

In [ ]:
lm = smf.ols('y~x', data).fit()
lm.summary()

The model fits the data very well. The estimated parameters a close to the true parameters.

### F. Plotting Estimate & Truth

Make a scatter plot of `y` versus `x` and overlay the *population regression line* and the *least squares line*. Create an appropriate legend by using labels and the `legend()` function. 

In [ ]:
ax = sns.scatterplot(x='x', y='y', data=data)
xs = np.linspace(*ax.get_xlim(), 10)
y_true = -1 + 0.5 * xs
y_fit = lm.predict({'x': xs})
ax.plot(xs, y_true, label='truth')
ax.plot(xs, y_fit, color='C1', label='fit')
ax.legend()
plt.show()

### G. Quadratic Term

Now fit a model that uses `x` and `x`${}^2$ as the predictors. Does the quadratic term improve the model fit? Explain you answer. 

In [ ]:
lm2 = smf.ols('y~x+I(x**2)', data).fit()
lm2.summary()

No, the quadratic term does *not* improve the model fit; the $F$-statistic is significantly lower. Also the $p$-value of the quadratic term is very high. This is of course expected, because the data has a truly linear relationship.

### H. Less Noise

Repeat E and F after creating a new data set with *less* noise. This can be achieved by creating the `eps` vector with a smaller standard deviation.

In [ ]:
eps = np.random.normal(0, 0.1, size=100)
y = -1 + 0.5 * x + eps
data_less = pd.DataFrame({'y': y, 'x':x})

In [ ]:
lm_less = smf.ols('y~x', data_less).fit()
lm_less.summary()

The model fit has improved, as evidenced by the larger $F$-statistic. The estimated parameters are very close to the true parameters.

In [ ]:
ax = sns.scatterplot(x='x', y='y', data=data_less)
xs = np.linspace(*ax.get_xlim(), 10)
y_true = -1 + 0.5 * xs
y_fit = lm_less.predict({'x': xs})
ax.plot(xs, y_true, label='truth')
ax.plot(xs, y_fit, color='C1', label='fit')
ax.legend()
plt.show()

The fitted line almost exactly matches the true line.

### I. More Noise

Repeat E and F after creating a new data set with *more* noise. This can be achieved by creating the `eps` vector with a larger standard deviation.

In [ ]:
eps = np.random.normal(0, 1.5, size=100)
y = -1 + 0.5 * x + eps
data_more = pd.DataFrame({'y': y, 'x':x})

In [ ]:
lm_more = smf.ols('y~x', data_more).fit()
lm_more.summary()

The model fit is considerably worse, as evidenced by the small $F$-statistic. The estimated parameters deviate much more from the true parameters.

In [ ]:
ax = sns.scatterplot(x='x', y='y', data=data_more)
xs = np.linspace(*ax.get_xlim(), 10)
y_true = -1 + 0.5 * xs
y_fit = lm_more.predict({'x': xs})
ax.plot(xs, y_true, label='truth')
ax.plot(xs, y_fit, color='C1', label='fit')
ax.legend()
plt.show()

The fitted line now deviates considerably from the true line. 

### J. Confidence Intervals

What are the confidence intervals for the
parameters $\beta_0$ and $\beta_1$ in the
three model fits? Comment on your results.

In [ ]:
print('Normal:\n', lm.conf_int())
print('\nLess noise:\n', lm_less.conf_int())
print('\nMore noise:\n', lm_more.conf_int())

The confidence intervals clearly depend on the noise: the more noise there is the wider they are, as expected.